In [19]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.applications.densenet import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor

%matplotlib inline

/Users/trupolog/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [20]:
data = pd.read_csv('../datasets/Processed_DJI.csv', index_col=[0], parse_dates=[0])
data = data[['Close','Volume']]

In [21]:
def make_features(data, max_lag, rolling_mean_size):
    data['dayofweek'] = data.index.dayofweek
    data['year'] = data.index.year
    data['month'] = data.index.month

    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['Close'].shift(lag)

    data['rolling_mean'] = data['Close'].shift().rolling(rolling_mean_size).mean()


In [22]:
make_features(data, 168, 168)
data

/var/folders/47/4y0ypxc92dz4v_h0jyn8jqpw0000gn/T/ipykernel_3174/615298662.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['lag_{}'.format(lag)] = data['Close'].shift(lag)
/var/folders/47/4y0ypxc92dz4v_h0jyn8jqpw0000gn/T/ipykernel_3174/615298662.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['lag_{}'.format(lag)] = data['Close'].shift(lag)
/var/folders/47/4y0ypxc92dz4v_h0jyn8jqpw0000gn/T/ipykernel_3174/615298662.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fr

,Close,Volume,dayofweek,year,month,lag_1,lag_2,lag_3,lag_4,lag_5,...,lag_160,lag_161,lag_162,lag_163,lag_164,lag_165,lag_166,lag_167,lag_168,rolling_mean
Date,,,,,,,,,,,,,,,,,,,,,
2009-12-31,10428.049805,NaN,3,2009,12,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04,10583.959961,NaN,0,2010,1,10428.049805,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,10572.019531,NaN,1,2010,1,10583.959961,10428.049805,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,10573.679688,0.515598,2,2010,1,10572.019531,10583.959961,10428.049805,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,10606.860352,9.776045,3,2010,1,10573.679688,10572.019531,10583.959961,10428.049805,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-11-09,23461.939453,6.511740,3,2017,11,23563.359375,23557.230469,23548.419922,23539.189453,23516.259766,...,20596.720703,20656.580078,20661.300781,20668.009766,20905.859375,20914.619141,20934.550781,20950.099609,20837.369141,21667.338728
2017-11-10,23422.210938,-0.991838,4,2017,11,23461.939453,23563.359375,23557.230469,23548.419922,23539.189453,...,20550.980469,20596.720703,20656.580078,20661.300781,20668.009766,20905.859375,20914.619141,20934.550781,20950.099609,21682.961170
2017-11-13,23439.699219,-65.347705,0,2017,11,23422.210938,23461.939453,23563.359375,23557.230469,23548.419922,...,20701.500000,20550.980469,20596.720703,20656.580078,20661.300781,20668.009766,20905.859375,20914.619141,20934.550781,21697.676119


In [23]:
data = data.drop('Volume', axis=1)

In [24]:
from sklearn.preprocessing import StandardScaler

train, test = train_test_split(data, test_size=0.1, random_state=12345, shuffle=False)
train = train.dropna()


features_train = train.drop('Close', axis=1)
target_train = train['Close']
features_test = test.drop('Close', axis=1)
target_test = test['Close']
scaler = StandardScaler()
scaler.fit(features_train)
features_train = scaler.transform(features_train)
features_test = scaler.transform(features_test)

In [25]:
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn.linear_model import LinearRegression

lin_model = LinearRegression()
lin_model.fit(features_train, target_train)
grid = cross_val_score(lin_model, X=features_train, y=target_train, n_jobs=-1, scoring='neg_root_mean_squared_error', cv=TimeSeriesSplit(n_splits=5))
print('RMSE на кросс-валидации', grid.mean())
grid = cross_val_score(lin_model, X=features_train, y=target_train, n_jobs=-1, scoring='r2', cv=TimeSeriesSplit(n_splits=5))
print('R2 на кросс-валидации', grid.mean())

RMSE на кросс-валидации -343.5608172433523
R2 на кросс-валидации 0.2756232792573243


In [ ]:
%%time

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor



model = RandomForestRegressor(random_state=12345)
params = {'n_estimators': range(100, 601, 50), 'max_depth': range(5, 51, 10)}
cv = TimeSeriesSplit(n_splits=5)
grid = GridSearchCV(model, params, n_jobs=-1, verbose=True, cv=cv, scoring='r2')
grid.fit(features_train, target_train)
print('Лучшие параметры:', grid.best_params_)
print('Лучший счет:', grid.best_score_)

In [ ]:

forest = RandomForestRegressor(max_depth=35, n_estimators=600, random_state=12345)
forest.fit(features_train, target_train)

In [ ]:
%%time


model = CatBoostRegressor(loss_function="MAPE")
params = {'learning_rate': [0.01, 0.03, 0.1, 0.12],
        'depth': [4, 5, 6, 7, 10, 20],
        'l2_leaf_reg': [1, 3, 5, 6,  7, 8, 9]}
grid_search_result = model.grid_search(params, X=features_train, y=target_train, cv = TimeSeriesSplit(n_splits=5))


In [0]:
print(grid_search_result["params"]["depth"])
print(grid_search_result["params"]["learning_rate"])
print(grid_search_result["params"]["l2_leaf_reg"])

In [29]:
best_depth = 6
best_learning_rate = 0.12 #ЛУЧШИЕ ДЛЯ МАПЕ
best_l2 = 3

In [30]:


def smape(A, F):
    return 100 / len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))


def metrics(y_true, y_pred):
    print('R2 Score:', r2_score(y_true=y_true, y_pred=y_pred))
    print('RMSE:', mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False))
    print('MAPE:', mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred))
    return r2_score(y_true=y_true, y_pred=y_pred), mean_squared_error(y_true=y_true, y_pred=y_pred, squared=False), mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred)

In [39]:
model_cat = CatBoostRegressor(depth=best_depth, learning_rate=best_learning_rate, l2_leaf_reg= best_l2, loss_function='MAPE')
model_cat.fit(features_train, target_train)
print('MAPE', mean_absolute_percentage_error(y_true=target_test, y_pred=model_cat.predict(features_test)))

0:	learn: 0.1460308	total: 14.8ms	remaining: 14.8s
1:	learn: 0.1320703	total: 22.8ms	remaining: 11.4s
2:	learn: 0.1188133	total: 29.3ms	remaining: 9.75s
3:	learn: 0.1127556	total: 39.3ms	remaining: 9.78s
4:	learn: 0.1073027	total: 46.3ms	remaining: 9.21s
5:	learn: 0.1014448	total: 52.4ms	remaining: 8.68s
6:	learn: 0.0926364	total: 59ms	remaining: 8.37s
7:	learn: 0.0885089	total: 65.9ms	remaining: 8.17s
8:	learn: 0.0820857	total: 72.4ms	remaining: 7.98s
9:	learn: 0.0797856	total: 78.1ms	remaining: 7.73s
10:	learn: 0.0744111	total: 83.6ms	remaining: 7.51s
11:	learn: 0.0717590	total: 89.2ms	remaining: 7.34s
12:	learn: 0.0697892	total: 95.5ms	remaining: 7.25s
13:	learn: 0.0667040	total: 101ms	remaining: 7.1s
14:	learn: 0.0647808	total: 106ms	remaining: 6.96s
15:	learn: 0.0621054	total: 111ms	remaining: 6.85s
16:	learn: 0.0607585	total: 117ms	remaining: 6.76s
17:	learn: 0.0588060	total: 123ms	remaining: 6.69s
18:	learn: 0.0579018	total: 129ms	remaining: 6.66s
19:	learn: 0.0565370	total: 136